<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Datencheck" data-toc-modified-id="Datencheck-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Datencheck</a></span></li></ul></div>

# Aufbereitung Abstimmungsdaten Grossrat
In diesem Notebook werden die Daten der verschiedenen Abstimmungsanlagen geprüft. 

In [2]:
# Die nötigen Bilbiotheken werden importiert. 
import pandas as pd
from os import listdir
from tqdm import tqdm
import numpy as np
from fuzzywuzzy import process

In [108]:
# Die Daten werden exportiert. 
df = pd.read_csv('daten/abstimmungen_grossrat.csv', index_col='Unnamed: 0')
df['datetime'] = pd.to_datetime(df['datetime'])
df['date'] = df['datetime'].apply(lambda x: x.date())

2021-06-04 17:05:37,680 [203785] WARNING  py.warnings:109: [JupyterRequire] /usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (3,9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,



## Datencheck

In [109]:
# Ein erster Blick auf die Daten. 
df.head()

,nachname,vorname,wohnort,partei,datetime,geschäftsname,stimme,res_ja,res_nein,res_enthaltung,res_keine_stimme,anlage,dateiname,register,date
1,Aebi,Markus,Hellsau,SVP / UDC,2018-06-04 15:47:00,#2017.RRGR.459_0,ja,137.0,1.0,1.0,NaN,rathaus,Sommersession 2018.xlsx,4.04.2018,2018-06-04
2,Aebischer,Verena,Riffenmatt,SVP / UDC,2018-06-04 15:47:00,#2017.RRGR.459_0,ja,137.0,1.0,1.0,NaN,rathaus,Sommersession 2018.xlsx,4.04.2018,2018-06-04
3,Amstutz,Madeleine,Schwanden-Sigriswil,SVP / UDC,2018-06-04 15:47:00,#2017.RRGR.459_0,ja,137.0,1.0,1.0,NaN,rathaus,Sommersession 2018.xlsx,4.04.2018,2018-06-04
4,Augstburger,Ueli,Gerzensee,SVP / UDC,2018-06-04 15:47:00,#2017.RRGR.459_0,ja,137.0,1.0,1.0,NaN,rathaus,Sommersession 2018.xlsx,4.04.2018,2018-06-04
5,Benoit,Roland,Corgémont,SVP / UDC,2018-06-04 15:47:00,#2017.RRGR.459_0,ja,137.0,1.0,1.0,NaN,rathaus,Sommersession 2018.xlsx,4.04.2018,2018-06-04


In [110]:
# Wie viele Einträge sind vorhanden? 
len(df)

303914

In [111]:
# Welche Zeitspanne wird abgedeckt? 
df['datetime'].describe()[['first', 'last']]

2021-06-04 17:05:43,153 [203785] WARNING  py.warnings:109: [JupyterRequire] <ipython-input-111-8911b8d542a4>:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df['datetime'].describe()[['first', 'last']]



first    2018-06-04 15:47:00
last     2021-03-22 15:55:40
Name: datetime, dtype: object

In [112]:
# Wie viele Stimmen (bzw. Enthaltungen) liegen maximal pro Abstimmung vor? 
df_temp = df.groupby(by=['geschäftsname', 'dateiname', 'register', 'datetime'])['stimme'].count().to_frame()
df_temp['stimme'].max()

161

In [114]:
# Wie viele Stimmen (bzw. Enthaltungen) liegen minimal pro Abstimmung vor? 
df_temp = df.groupby(by=['geschäftsname', 'dateiname', 'register', 'date'])['stimme'].count().to_frame()
df_temp['stimme'].min()

116

In [115]:
# Welche Parteien sind verzeichnet? 
df['partei'].unique()

array(['SVP / UDC', 'SP-JUSO-PSA / PS-JS-PSA', 'FDP / PLR',
       'Grüne / Les Verts', 'BDP / PBD', 'glp / pvl', 'EVP / PEV',
       'EDU / UDF', 'CVP / PDC', 'SVP', 'EDU', 'FDP', 'BDP', 'glp',
       'Grüne', 'SP', 'EVP', 'PLR', 'PSA', 'AL', 'CVP', 'PS', 'les Verts',
       'UDC', 'PEV', 'PDC', 'SVP/UDC', 'FDP/PLR', 'Grüne/les Verts',
       'fraktionslos', 'Die Mitte', 'glp/plv', 'EVP/PEV', 'EDU/UDF'],
      dtype=object)

In [122]:
# Wie viele Parlamentarierinnen und Parlamentarier gibts pro Partei am ersten Tag? 
df[df['date'] == df['date'].min()][['nachname', 'vorname', 'partei']].drop_duplicates().groupby(by='partei').count()[['nachname']]

,nachname
partei,
BDP / PBD,14
EDU / UDF,5
EVP / PEV,10
FDP / PLR,20
Grüne / Les Verts,15
SP-JUSO-PSA / PS-JS-PSA,40
SVP / UDC,45
glp / pvl,11


In [121]:
# Wie viele verschiedene Personen sind verzeichnet? (Parteiwechsler zählen doppelt.)
len(df[['nachname', 'vorname', 'partei']].drop_duplicates())

463

In [ ]:
# Wie viele Stimmen gabs in den BEA1-Abstimmungen? 
df_temp = df_bea_1.groupby(by=['datetime', 'geschäftsname', 'dateiname', 'register'])['stimme'].count().to_frame()
df_temp.describe()

In [125]:
# An wievielen Abstimmungen haben die Parlamentarier teilgenommen?
df_temp = df.groupby(by=['nachname', 'vorname', 'partei'])['nachname'].count().to_frame()
df_temp.describe()

,nachname
count,460.000000
mean,657.317391
std,445.122792
min,1.000000
25%,258.000000
50%,451.000000
75%,1211.000000
max,1469.000000
